# Calvin-Benson-Bassham cycle model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from model import get_model
from modelbase.ode import Simulator
from modelbase.utils.plotting import plot_grid

In [ ]:
y0 = {
    "PGA": 0.6437280277346407,
    "BPGA": 0.001360476366780556,
    "GAP": 0.011274125311289358,
    "DHAP": 0.24803073890728228,
    "FBP": 0.019853938009873073,
    "F6P": 1.0950701164493861,
    "G6P": 2.5186612678035734,
    "G1P": 0.14608235353185037,
    "SBP": 0.09193353265673603,
    "S7P": 0.23124426886012006,
    "E4P": 0.028511831060903877,
    "X5P": 0.036372985623662736,
    "R5P": 0.06092475016463224,
    "RUBP": 0.24993009253928708,
    "RU5P": 0.02436989993734177,
    "ATP": 0.43604115800259613,
}

s = Simulator(get_model())
s.initialise(y0)
t, y = s.simulate(100_000)
groups = [
    ["G6P", "PGA", "F6P", "S7P"],
    ["RU5P", "X5P", "SBP", "G1P"],
    ["ATP", "DHAP", "RUBP", "R5P"],
    ["BPGA", "E4P", "FBP", "GAP"],
]

s.plot_grid(
    compound_groups=groups,
    ncols=2,
    sharex=True,
    sharey=False,
    xlabels="Time [au]",
    ylabels="Concentration [mM]",
    figure_title="Calvin cycle substrate concentrations",
)
plt.show()

In [ ]:
external_phosphate_scan = s.parameter_scan(
    parameter_name="Phosphate_pool_ext",
    parameter_values=np.linspace(0.1, 2, 128),
)

In [ ]:
result = external_phosphate_scan.copy()
time = result.index.values
result["BPGA"] *= 50

plot_groups = [(time, result.loc[:, i].values) for i in groups]
legend_groups = [[i if i != "BPGA" else "50x BPGA" for i in j] for j in groups]

fig, ax = plot_grid(
    plot_groups=plot_groups,
    legend_groups=legend_groups,
    ncols=2,
    sharey=False,
    xlabels="External orthophosphate [mM]",
    ylabels="Concentration [mM]",
)

## Dynamic NADPH variant

In [ ]:
def NADP_mod(NADPH, N_total):
    return [N_total - NADPH]


def v1(
    RUBP,
    PGA,
    FBP,
    SBP,
    P,
    NADPH,
    V1,
    Km1,
    Ki11,
    Ki12,
    Ki13,
    Ki14,
    Ki15,
):
    return (V1 * RUBP) / (
        RUBP
        + Km1
        * (1 + (PGA / Ki11) + (FBP / Ki12) + (SBP / Ki13) + (P / Ki14) + (NADPH / Ki15))
    )


def v3(BPGA, NADPH, GAP, P_i, NADP, protonsStroma, kRE, q3):
    return kRE * ((NADPH * BPGA * protonsStroma) - (GAP * NADP * P_i) / q3)


def vNADPH(NADP, Vmax_NADPH, Km_NADPH):
    return (Vmax_NADPH * NADP) / (Km_NADPH + NADP)

In [ ]:
m2 = get_model()
m2.remove_reactions(["v1", "v3"])
m2.remove_parameters(["NADPH_pool", "NADP_pool"])

m2.add_parameters({"Vmax_NADPH": 2.816, "Km_NADPH": 0.19})
m2.add_compound("NADPH")


m2.add_algebraic_module(
    module_name="NADH_mod",
    function=NADP_mod,
    compounds=["NADPH"],
    derived_compounds=["NADP"],
    parameters=["N_total"],
)


m2.add_reaction(
    rate_name="v1",
    function=v1,
    stoichiometry={"RUBP": -1, "PGA": 2},
    modifiers=["PGA", "FBP", "SBP", "Phosphate_pool", "NADPH"],
    parameters=[
        "Vmax_1",
        "Km_1",
        "Ki_1_1",
        "Ki_1_2",
        "Ki_1_3",
        "Ki_1_4",
        "Ki_1_5",
    ],
)


m2.add_reaction(
    rate_name="v3",
    function=v3,
    stoichiometry={"BPGA": -1, "NADPH": -1, "GAP": 1},
    modifiers=["Phosphate_pool", "NADP"],
    parameters=[
        "proton_pool_stroma",
        "k_rapid_eq",
        "q3",
    ],
    reversible=True,
)

m2.add_reaction(
    rate_name="vNADPH",
    function=vNADPH,
    stoichiometry={"NADPH": 1},
    modifiers=["NADP"],
    parameters=["Vmax_NADPH", "Km_NADPH"],
)

In [ ]:
y0_2 = {
    "PGA": 0.599645270373,
    "BPGA": 0.000907499521924,
    "GAP": 0.011839616887,
    "DHAP": 0.260471552645,
    "FBP": 0.021895569623,
    "F6P": 1.2456290719,
    "G6P": 2.86494686535,
    "G1P": 0.166166918189,
    "SBP": 0.1120019621,
    "S7P": 0.233467059202,
    "E4P": 0.0330766864679,
    "X5P": 0.0374527459593,
    "R5P": 0.0627333486958,
    "RUBP": 0.261466058509,
    "RU5P": 0.0250933393445,
    "ATP": 0.414993685612,
    "NADPH": 0.281543418344,
}

s2 = Simulator(m2)
s2.initialise(y0_2)
t, y = s2.simulate(200)
groups_nadph = [
    ["G6P", "PGA", "F6P", "S7P"],
    ["RU5P", "X5P", "SBP", "G1P"],
    ["ATP", "DHAP", "RUBP", "R5P", "NADPH"],
    ["BPGA", "E4P", "FBP", "GAP"],
]

s2.plot_grid(
    compound_groups=groups,
    ncols=2,
    sharex=True,
    sharey=False,
    xlabels="Time [au]",
    ylabels="Concentration [mM]",
    figure_title="Calvin cycle substrate concentrations",
)
plt.show()

In [ ]:
external_phosphate_scan_2 = s2.parameter_scan(
    parameter_name="Phosphate_pool_ext",
    parameter_values=np.linspace(0.1, 2, 128),
)

In [ ]:
result = external_phosphate_scan_2.copy()
time = result.index.values
result["BPGA"] *= 50

plot_groups = [(time, result.loc[:, i].values) for i in groups_nadph]
legend_groups = [[i if i != "BPGA" else "50x BPGA" for i in j] for j in groups_nadph]

fig, ax = plot_grid(
    plot_groups=plot_groups,
    legend_groups=legend_groups,
    ncols=2,
    sharey=False,
    xlabels="External orthophosphate [mM]",
    ylabels="Concentration [mM]",
)